In [1]:
# Reference: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_folding.ipynb#scrollTo=eb29483f

! pip install --upgrade transformers py3Dmol accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [1]:
from transformers import AutoTokenizer, EsmForProteinFolding
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)

model = model.cuda()
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForProteinFolding(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 2560, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 2560, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-35): 36 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((2560,), eps=1

In [2]:
import torch

torch.backends.cuda.matmul.allow_tf32 = True

In [3]:
model.trunk.set_chunk_size(64)

In [4]:
import pandas as pd

df = pd.read_csv("cleaned_liver_data_cdna.csv")
print(df["protein_length"].max())
df

7081


,protein_id,mrna_id,gene_id,protein_sequence,protein_length,cdna,5utr,cds,3utr,cdna_length,5utr_length,cds_length,3utr_length,protein_abundance,median_TPM,protein_per_transcript
0,ENSP00000341562,ENST00000341156,ENSG00000135454,MWLGRRALCALVLLLACASLGLLYASTRDAPGLRLPLAPWAPPQSP...,533,GCATTCCCCGCGCGGAGCCGAAGCAGCCGCAACGAGCCGGGAGCTG...,GCATTCCCCGCGCGGAGCCGAAGCAGCCGCAACGAGCCGGGAGCTG...,ATGTGGCTGGGCCGCCGGGCCCTGTGCGCTCTGGTCCTTCTGCTCG...,TGGCCCGCTGGGGATTTCTGACTGTCAGGCTGGGCCTGCCTCCTTG...,5368,431,1602,3335,0.068,0.100,0.680000
1,ENSP00000290765,ENST00000290765,ENSG00000133433,MGLELFLDLVSQPSRAVYIFAKKNGIPLELRTVDLVKGQHKSKEFL...,244,AGCTGCTGCCCACACCGCGCTCAGCGCCTTCACTGCCATCCCCGCT...,AGCTGCTGCCCACACCGCGCTCAGCGCCTTCACTGCCATCCCCGCT...,ATGGGCCTAGAGCTGTTTCTTGACCTGGTGTCCCAGCCCAGCCGCG...,AGGGTCTGGGATGGGGGCCAGGAGATTAGCAACAAGGATTCATTCT...,1108,64,735,309,57.000,11.385,5.006588
2,ENSP00000256996,ENST00000256996,ENSG00000134574,MAPKKRPETQKTSEIVLRPRNKRSRSPLELEPEAKKLCAKGSGPSR...,427,GGTTTGAACAAGCCCTGGGCATGTTTGGCGGGAAGTTGGCTTAGCT...,GGTTTGAACAAGCCCTGGGCATGTTTGGCGGGAAGTTGGCTTAGCT...,ATGGCTCCCAAGAAACGCCCAGAAACCCAGAAGACCTCCGAGATTG...,GAGACACTAAAGAAGGTGTGGGCCAGACAAGGCCTTGGAGCCCACA...,1815,163,1284,368,2.510,3.740,0.671123
3,ENSP00000262990,ENST00000262990,ENSG00000109445,MPKKKTGARKKAENRREREKQLRASRSTIDLAKHPCNASMECDKCQ...,320,GCTGTCGTAAAAGGACGTCCGGTCCGTCTCCTAGTGTCCGGAATCG...,GCTGTCGTAAAAGGACGTCCGGTCCGTCTCCTAGTGTCCGGAATCG...,ATGCCTAAAAAAAAGACTGGTGCGAGGAAGAAGGCTGAGAACCGCC...,GGGAGCTGCTCTGGTGGCCGTGTGTGAGAGGAGCAGGAGTGAGTGT...,1891,220,963,708,2.500,13.445,0.185943
4,ENSP00000371811,ENST00000382374,ENSG00000165487,MAAAAGSCARVAAWGGKLRRGLAVSRQAVRSPGPLAAAVAGAALAG...,434,GCCTAGCTGCGCTTCCGCAAAGATGGCGGCGGCTGCGGGTAGCTGC...,GCCTAGCTGCGCTTCCGCAAAG,ATGGCGGCGGCTGCGGGTAGCTGCGCGCGGGTGGCGGCCTGGGGCG...,TAAAAGATATAATAGTATGGCAATTATATTGTTCCAAATGTCAAAA...,1885,22,1305,558,2.500,8.060,0.310174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,ENSP00000387111,ENST00000410042,ENSG00000273155,MAGILRLVVQWPPGRLQTVTKGVESLICTDWIRHKFTRSRIPEKVF...,131,GCACTTTTCCAGCTCGAGCCCTCACGAGGCCGTGGGTACGACCGGA...,GCACTTTTCCAGCTCGAGCCCTCACGAGGCCGTGGGTACGACCGGA...,ATGGCTGGGATTTTGCGCTTAGTAGTTCAATGGCCCCCAGGCAGAC...,TGTAATTCTAAATGCATTTTTCTTTTTTCGTTAATGTTCTGAAAAA...,669,207,396,66,1.010,0.155,6.516129
7983,ENSP00000477037,ENST00000553909,ENSG00000259171,MVMGLGVLLLVFVLGLGLTPPTLAQDNSRHL,31,ACACACTCACACAAGGACGCCAACCCCACCTAGATGCAAAGCAGGA...,ACACACTCACACAAGGACGCCAACCCCACCTAGATGCAAAGCAGGA...,ATGGTGATGGGCCTGGGCGTTTTGTTGTTGGTCTTCGTGCTGGGTC...,GATACTGATGGCTCTGCAGAGGACCCATTCATTGCTTCTGCTTTTG...,1498,190,96,1212,0.591,24.155,0.024467
7984,ENSP00000361601,ENST00000372523,ENSG00000168612,MLERLKAPWSAALQRKYFDLGIWTAPISPMALTMLNGLLIKDSSPP...,485,AAATAGGGAGAAATGGCGACGGAGCCTGGCTGTGGGCCCATCTTTG...,AAATAGGGAGAAATGGCGACGGAGCCTGGCTGTGGGCCCATCTTTG...,ATGCTTGAGAGACTCAAAGCCCCGTGGTCAGCTGCCCTGCAAAGAA...,TTATTCTCGATGCCCAGAGATGCTCATGCACCTGTGCACACTCACA...,2769,95,1458,1216,0.030,2.750,0.010909
7985,ENSP00000437563,ENST00000539033,ENSG00000255641,MNKQRGTFSEVSLAQDPKRQQRKPKGNKSSISGTEQEIFQVELNLQ...,240,CACACAGCTGCAGAGATGAATAAACAAAGAGGAACCTTCTCAGAAG...,CACACAGCTGCAGAG,ATGAATAAACAAAGAGGAACCTTCTCAGAAGTGAGTCTGGCCCAGG...,GCTCAAGAAATCAACACATCTTGGCCTCCCAAGTTGCTGGGATTAC...,998,15,723,260,0.102,0.110,0.927273


In [5]:
sequences = df["protein_sequence"].to_list()
sequence_ids = df["protein_id"].to_list()

In [6]:

# Print model's top-level attributes
print("Model attributes:")
for name, module in model.named_children():
    print(f"- {name}")

# Specifically look at the trunk module
print("\nTrunk module attributes:")
for name, module in model.trunk.named_children():
    print(f"- {name}")

Model attributes:
- esm
- esm_s_mlp
- embedding
- trunk
- distogram_head
- ptm_head
- lm_head
- lddt_head

Trunk module attributes:
- pairwise_positional_embedding
- blocks
- recycle_s_norm
- recycle_z_norm
- recycle_disto
- structure_module
- trunk2sm_s
- trunk2sm_z


In [6]:
test_protein = "MGAGASAEEKHSRELEKKLKEDAEKDARTVKLLLLGAGESGKSTIVKQMKIIHQDGYSLEECLEFIAIIYGNTLQSILAIVRAMTTLNIQYGDSARQDDARKLMHMADTIEEGTMPKEMSDIIQRLWKDSGIQACFERASEYQLNDSAGYYLSDLERLVTPGYVPTEQDVLRSRVKTTGIIETQFSFKDLNFRMFDVGGQRSERKKWIHCFEGVTCIIFIAALSAYDMVLVEDDEVNRMHESLHLFNSICNHRYFATTSIVLFLNKKDVFFEKIKKAHLSICFPDYDGPNTYEDAGNYIKVQFLELNMRRDVKEIYSHMTCATDTQNVKFVFDAVTDIIIKENLKDCGLF"

tokenized_input = tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)['input_ids']
tokenized_input = tokenized_input.cuda()

import torch

with torch.no_grad():
    output = model(tokenized_input)

output['s_z']

tensor([[[[-1.5312e+01, -1.1654e+01,  2.0418e+01,  ...,  7.2015e+01,
            6.5953e+01,  9.4722e+00],
          [ 2.0443e+01, -1.7881e+01,  9.9735e+00,  ...,  5.0363e+01,
            2.2189e+01,  7.3537e+00],
          [-1.4686e+01, -8.0592e+00, -1.6122e+00,  ...,  5.5027e+01,
            2.4880e+01, -1.3444e+01],
          ...,
          [ 7.1311e+00, -5.6107e+00, -1.3823e+01,  ..., -1.4397e+01,
            1.6523e+01,  2.6203e+00],
          [ 6.6875e+00, -6.1878e+00, -1.0535e+01,  ..., -2.3282e+01,
            8.6818e+00, -2.3309e+00],
          [ 5.0486e+00, -7.3074e+00, -7.5575e+00,  ..., -6.2829e+00,
            8.7185e+00, -2.8480e+00]],

         [[ 1.2287e+01, -1.7487e+01, -2.2553e+01,  ...,  6.1522e+01,
            2.7221e+01,  4.5899e+00],
          [-1.2862e+01, -4.9824e+00,  4.8187e+00,  ...,  9.2095e+01,
            7.0474e+01,  3.1940e+00],
          [ 3.7239e+01, -3.9221e+01, -1.2296e+01,  ...,  6.0434e+01,
            3.6214e+01, -7.5250e-01],
          ...,
     

In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc
import os


def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize()


def batch_sequences(sequences, sequence_ids, batch_size=8):
    for i in range(0, len(sequences), batch_size):
        yield sequences[i : i + batch_size], sequence_ids[i : i + batch_size]


embeddings_list = []
processed_ids = []
skipped_sequences = []

batch_size = 4
total_batches = (len(sequences) + batch_size - 1) // batch_size

clear_gpu_memory()

for batch_sequences, batch_ids in tqdm(
    batch_sequences(sequences, sequence_ids, batch_size),
    total=total_batches,
    desc="Processing batches",
):
    try:
        clear_gpu_memory()

        tokenized_input = tokenizer(
            batch_sequences,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
            add_special_tokens=False,
        )["input_ids"]

        # Get attention mask for proper mean pooling
        attention_mask = (tokenized_input != tokenizer.pad_token_id).float()

        tokenized_input = tokenized_input.cuda()
        attention_mask = attention_mask.cuda()

        with torch.no_grad():
            # ESM-2 processing
            esm_output = model.esm(tokenized_input)
            last_hidden_state = esm_output.last_hidden_state.clone()
            del esm_output
            clear_gpu_memory()

            # ESM-S projection layer
            esm_s = model.esm_s_mlp(last_hidden_state)
            del last_hidden_state
            clear_gpu_memory()

            # Project to structure module dimension (1024 -> 384)
            structure_embeddings = model.trunk.trunk2sm_s(
                esm_s
            )  # [batch_size, seq_len, 384]
            del esm_s
            clear_gpu_memory()

            # Apply mean pooling
            masked_embeddings = structure_embeddings * attention_mask.unsqueeze(-1)
            del structure_embeddings
            clear_gpu_memory()

            # Sum and divide by the number of actual tokens
            sum_embeddings = masked_embeddings.sum(dim=1)  # [batch_size, 384]
            del masked_embeddings
            clear_gpu_memory()

            seq_lengths = attention_mask.sum(dim=1, keepdim=True)  # [batch_size, 1]
            del attention_mask
            clear_gpu_memory()

            mean_pooled_embeddings = sum_embeddings / seq_lengths
            del sum_embeddings, seq_lengths
            clear_gpu_memory()

            # Convert to numpy and store
            structure_embeddings_np = mean_pooled_embeddings.cpu().numpy()
            del mean_pooled_embeddings
            clear_gpu_memory()

            # Store embeddings and IDs
            embeddings_list.append(structure_embeddings_np)
            processed_ids.extend(batch_ids)

            del structure_embeddings_np
            clear_gpu_memory()

    except Exception as e:
        print(f"Error processing batch: {str(e)}")
        skipped_sequences.extend(batch_ids)
        clear_gpu_memory()
        continue

    if len(embeddings_list) % 100 == 0:
        # Save temporary results
        temp_embeddings = np.concatenate(embeddings_list, axis=0)
        temp_df = pd.DataFrame(
            temp_embeddings, columns=[f"esm_struct_{i}" for i in range(384)]
        )
        temp_df.insert(0, "id", processed_ids)
        temp_df.to_csv(
            f"structure_embeddings_temp_{len(processed_ids)}.csv", index=False
        )
        del temp_df, temp_embeddings
        clear_gpu_memory()

try:
    all_embeddings = np.concatenate(embeddings_list, axis=0)

    embedding_columns = [f"esm_struct_{i}" for i in range(384)]

    df_structure = pd.DataFrame(all_embeddings, columns=embedding_columns)
    df_structure.insert(0, "id", processed_ids)

    print(f"Successfully processed {len(processed_ids)} sequences")
    print(f"Skipped {len(skipped_sequences)} sequences")
    print(f"DataFrame shape: {df_structure.shape}")

    df_structure.to_csv("prot_structure_embeddings.csv", index=False)

    for file in os.listdir():
        if file.startswith("structure_embeddings_temp_"):
            os.remove(file)

except Exception as e:
    print(f"Error in final processing: {str(e)}")
    print("Attempting to save partial results...")
    for i, (emb_batch, ids_batch) in enumerate(
        zip(
            embeddings_list,
            [
                processed_ids[i : i + batch_size]
                for i in range(0, len(processed_ids), batch_size)
            ],
        )
    ):
        temp_df = pd.DataFrame(
            emb_batch, columns=[f"esm_struct_{i}" for i in range(384)]
        )
        temp_df.insert(0, "id", ids_batch)
        temp_df.to_csv(f"structure_embeddings_partial_{i}.csv", index=False)

finally:
    # Clear all remaining variables
    del embeddings_list, processed_ids, skipped_sequences
    clear_gpu_memory()

Processing batches:   3%|▎         | 51/1997 [02:38<1:16:03,  2.35s/it]

Error processing batch: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).


Processing batches:  16%|█▌        | 311/1997 [16:11<1:07:06,  2.39s/it]

Error processing batch: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).


Processing batches:  20%|██        | 401/1997 [20:52<1:23:54,  3.15s/it]

...

Error processing batch: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).


Processing batches:  94%|█████████▎| 1868/1997 [1:37:14<05:03,  2.35s/it]

Error processing batch: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).


Processing batches: 100%|██████████| 1997/1997 [1:44:05<00:00,  3.13s/it]


Successfully processed 7927 sequences
Skipped 60 sequences
DataFrame shape: (7927, 385)


In [20]:
tokenizer

EsmTokenizer(name_or_path='facebook/esmfold_v1', vocab_size=26, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': 'A', 'cls_token': '<cls>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("A", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	22: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	23: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	25: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	26: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

# ESMFold and ESM-2

Lin, Z., Akin, H., Rao, R., Hie, B., Zhu, Z., Lu, W., Smetanin, N., Verkuil, R., Kabeli, O., Shmueli, Y., Costa, A. D. S., Fazel-Zarandi, M., Sercu, T., Candido, S., & Rives, A. (2023). Evolutionary-scale prediction of atomic-level protein structure with a language model. Science, 379(6637), 1123–1130. https://doi.org/10.1126/science.ade2574
